Do in shell :

`mkdir tmpzip && mkdir tmp && mkdir data && cd data`

`wget -r --no-parent https://data.cquest.org/inpi_rncs/imr/`

`mkdir data.cquest.org/inpi_rncs/imr/flux`

`mv data.cquest.org/inpi_rncs/imr/20* data.cquest.org/inpi_rncs/imr/flux/`

les flux sont ok mis il faut les dezippé : `for f in ./*7z; do 7z x $f; done`
supprimer tous les fichiers  md5

Passons aux stocks :

2017:
`for f in *.zip; do mkdir ${f:0:4} && mkdir ${f:0:4}/${f:5:2} && unzip $f -d ${f:0:4}/${f:5:2}/; done && rm *.zip && rm *.html;`

2018 ok

2019, 2020, 2021, 2022
`for f in */*/*.zip; do mkdir ${f:0:2}/${f:3:2}/${f:6:4} && mkdir ${f:0:2}/${f:3:2}/${f:6:4}/${f:11:2} && unzip $f -d ${f:0:2}/${f:3:2}/${f:6:4}/${f:11:2}/; done`
`rm */*/*.html && rm */*/*.zip && rm */*/*.md5`

On ordonne le tout pour que toutes les données soient dans data/TYPE_DONNEES/ANNEE/MOIS/JOUR/GREFFECODE/GREFFECODE2/DONNEESCSV

In [1]:
import sqlite3
import shutil
import zipfile
import glob
import unidecode
import os
import py7zr
from datetime import date, timedelta
import pandas as pd
import time
from os.path import exists


In [2]:
sdate = date(2017,5,4)   # start date
#sdate = date(2022,6,29)   # start date
edate = date(2022,9,3)   # end date
list_dates = [x.strftime("%Y-%m-%d") for x in pd.date_range(sdate,edate-timedelta(days=1),freq='d').to_list()]

In [3]:
vol_data = '/srv/geoff/inpi/data/'
vol_data = '/Volumes/OnlyData/tmp/'

In [4]:
# Etape de processing des données en amont du lancement de l'enrichissement de la bdd sqlite
def concatFilesRep(type_file, name_concat, pattern):
    pathdate = vol_data + type_file + "/" + ld.split('-')[0] + '/' + ld.split('-')[1] + '/' + ld.split('-')[2] + '/'
    consofile = vol_data + 'synthese/' + ld + name_concat + '.csv'
    # get list files stock
    list_files = glob.glob(pathdate + pattern)
    list_files.sort()
    if len(list_files) > 0:
        
        with open(consofile,"wb") as fout:
            # first file:
            with open(list_files[0], "rb") as f:
                fout.write(f.read())
            # now the rest:    
            for ls in list_files[1:]:
                with open(ls, "rb") as f:
                    try:
                        next(f) # skip the header
                    except:
                        pass
                    fout.write(f.read())
        
        
        #os.system('rm ' + consofile + ' && ulimit -n 10240 && head -n1 ' + list_files[0] + ' > ' + consofile)
        #for ls in list_files:
        #    os.system('tail -n+2 -q ' + ls + ' >> ' + consofile)
        
        print(name_concat + " ok")

In [9]:
for ld in list_dates:
    print(ld)
    concatFilesRep("stock", "_stock_rep", "*/*/*_5_rep.csv")
    concatFilesRep("flux-tc", "_flux_rep", "*/*/*_5_rep.csv")
    concatFilesRep("flux-tc", "_flux_rep_nouveau_modifie", "*/*/*6_rep_nouveau_modifie_EVT.csv")
    concatFilesRep("flux-tc", "_flux_rep_partant", "*/*/*7_rep_partant_EVT.csv")
    

2022-05-31
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-01
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-02
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-03
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-04
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-05
2022-06-06
2022-06-07
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-08
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-09
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-10
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-11
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-12
2022-06-13
2022-06-14
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-15
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-16
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_parta

In [54]:
!rm inpi.db

In [55]:
connection = sqlite3.connect('inpi.db')

In [56]:
cursor = connection.cursor()

In [57]:
cursor.execute('''CREATE TABLE IF NOT EXISTS rep_pp
              (siren TEXT, nom_patronymique TEXT, nom_usage TEXT, prenoms TEXT, datenaissance TEXT, villenaissance TEXT, paysnaissance TEXT, qualite TEXT)''')


In [58]:
cursor.execute('''CREATE TABLE IF NOT EXISTS rep_pm
              (siren TEXT, denomination TEXT, siren_pm TEXT, sigle TEXT, qualite TEXT)''')

In [59]:
cursor.execute('''CREATE INDEX idx_siren_pp ON rep_pp (siren);''')
cursor.execute('''CREATE INDEX idx_siren_pm ON rep_pm (siren);''')
cursor.execute('''CREATE INDEX idx_nom_patronymique_pp ON rep_pp (nom_patronymique);''')
cursor.execute('''CREATE INDEX idx_nom_usage_pp ON rep_pp (nom_usage);''')
cursor.execute('''CREATE INDEX idx_siren_pm_pm ON rep_pm (siren_pm);''')

In [60]:
connection.commit()

In [61]:
def uniformizeDf(df):
    for c in df.columns:
        df = df.rename(columns={c: unidecode.unidecode(
                                c.lower() \
                                .replace(' ', '') \
                                .replace('.','') \
                                .replace('_','') \
                                .replace('"','') \
                                .replace("'",'')
                    )})
    if 'sigle' not in df.columns:
        df['sigle'] = ''
    # Do something
    df['siren'] = df.fillna('')['siren']
    df['nom_patronymique'] = df.fillna('')['nompatronymique']
    df['nom_usage'] = df.fillna('')['nomusage']
    df['prenoms'] = df.fillna('')['prenoms']
    df['nom_patronymique'] = df['nom_patronymique'].apply(lambda x: str(x).replace(',','').lower())
    df['nom_usage'] = df['nom_usage'].apply(lambda x: str(x).replace(',','').lower())
    df['prenoms'] = df['prenoms'].apply(lambda x: str(x).replace(',','').lower())
    df['datenaissance'] = df.fillna('')['datenaissance']
    df['villenaissance'] = df.fillna('')['villenaissance']
    df['paysnaissance'] = df.fillna('')['paysnaissance']
    df['denomination'] = df.fillna('')['denomination']
    df['siren_pm'] = df.fillna('')['siren1']
    df['sigle'] = df.fillna('')['sigle']
    df['qualite'] = df.fillna('')['qualite']
    dfpp = df[df['type'].str.contains('Physique')][['siren', 'nom_patronymique', 'nom_usage','prenoms', 'datenaissance', 'villenaissance', 'paysnaissance', 'qualite']]
    dfpm = df[df['type'].str.contains('Morale')][['siren','denomination', 'siren_pm', 'sigle', 'qualite']]

    return dfpp, dfpm


In [62]:
# On s'occupe du stock initial (car vraiment gros et pas adapté à la méthode ci-dessous)
list_files = glob.glob(vol_data + 'stock/2017/05/04/*/*/*5_rep.csv')
list_files.sort()
for lf in list_files:
    print(lf)
    df = pd.read_csv(lf,sep=";",dtype=str)
    dfpp, dfpm = uniformizeDf(df)
    dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
    dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')

/Volumes/OnlyData/tmp/stock/2017/05/04/0101/S1/0101_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0202/S1/0202_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0203/S1/0203_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0301/S1/0301_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0303/S1/0303_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0401/S1/0401_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0501/S1/0501_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0601/S1/0601_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0602/S1/0602_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0603/S1/0603_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0605/S1/0605_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0702/S1/0702_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0802/S1/0802_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04

/Volumes/OnlyData/tmp/stock/2017/05/04/7702/S1/7702_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/7801/S1/7801_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/7802/S1/7802_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/7803/S1/7803_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/7901/S1/7901_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8002/S1/8002_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8101/S1/8101_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8102/S1/8102_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8201/S1/8201_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8302/S1/8302_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8303/S1/8303_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8305/S1/8305_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8401/S1/8401_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04

In [63]:
original = 'inpi.db'
target = 'inpi-stock-initial.db'
shutil.copyfile(original, target)

'inpi-stock-initial.db'

In [64]:
list_dates[2]

'2017-05-06'

In [65]:
len(list_dates)

1948

In [ ]:
# Toutes les dates après la date du stock initial
for ld in list_dates[1:]:
    print(ld)
    #if((ld.split('-')[1] == '01') & (ld.split('-')[2] == '01')):
    if(ld.split('-')[2] == '01'):
        original = 'inpi.db'
        target = 'inpi-'+ld+'.db'
        shutil.copyfile(original, target)
    
    consofile = vol_data + 'synthese/' + ld + '_stock_rep.csv'
    # Manage stocks
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        print('loaded and uniformize')
        # Delete old stock
        if(ld != '2017-05-04'):
            print('Old stock')
            result = 0
            for index,row in dfpp.iterrows():
                del_query = """DELETE from rep_pp where siren = ?"""
                cursor.execute(del_query, (row['siren'],))
                result = result + cursor.rowcount
            connection.commit()
            for index,row in dfpp.iterrows():
                del_query = """DELETE from rep_pm where siren = ?"""
                cursor.execute(del_query, (row['siren'],))
                result = result + cursor.rowcount
            connection.commit()
            print('Old Stock deleted : ' + str(result) + ' deleted records')
        # Add new stock
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Stock processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    
    consofile = vol_data + 'synthese/' + ld + '_flux_rep.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Flux rep processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    consofile = vol_data + 'synthese/' + ld + '_flux_rep_nouveau_modifie.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Flux rep modified new processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    
    consofile = vol_data + 'synthese/' + ld + '_flux_rep_partant.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        # Delete each rep
        result = 0
        for index,row in dfpp.iterrows():
            del_query = """DELETE from rep_pp where siren = ? AND nom_patronymique = ? AND prenoms = ? AND qualite = ?"""
            cursor.execute(del_query, (row['siren'], row['nom_patronymique'], row['prenoms'], row['qualite']))
            result = result + cursor.rowcount
        connection.commit()
        for index,row in dfpm.iterrows():
            del_query = """DELETE from rep_pm where siren = ? AND siren_pm = ? AND qualite = ?"""
            cursor.execute(del_query, (row['siren'], row['siren_pm'], row['qualite']))
            result = result + cursor.rowcount
        connection.commit()
        print('Flux rep partant processed : ' + str(result) + ' deleted records')

        


2017-05-05
2017-05-06
2017-05-07
2017-05-08
2017-05-09
2017-05-10
2017-05-11
2017-05-12
2017-05-13
2017-05-14
2017-05-15
2017-05-16
2017-05-17
2017-05-18
Flux rep processed : 20099added records
Flux rep modified new processed : 19800added records
Flux rep partant processed : 5687 deleted records
2017-05-19
2017-05-20
2017-05-21
2017-05-22
2017-05-23
2017-05-24
Flux rep processed : 23848added records
Flux rep modified new processed : 20330added records
Flux rep partant processed : 5698 deleted records
2017-05-25
Flux rep processed : 3914added records
Flux rep modified new processed : 2605added records
Flux rep partant processed : 782 deleted records
2017-05-26
Flux rep processed : 468added records
Flux rep modified new processed : 15added records
Flux rep partant processed : 1 deleted records
2017-05-27
Flux rep processed : 2760added records
Flux rep modified new processed : 1417added records
Flux rep partant processed : 357 deleted records
2017-05-28
2017-05-29
2017-05-30
Flux rep proc

Flux rep partant processed : 1181 deleted records
2017-08-02
Flux rep processed : 4050added records
Flux rep modified new processed : 3328added records
Flux rep partant processed : 1267 deleted records
2017-08-03
Flux rep processed : 4285added records
Flux rep modified new processed : 3326added records
Flux rep partant processed : 1137 deleted records
2017-08-04
Flux rep processed : 4774added records
Flux rep modified new processed : 5149added records
Flux rep partant processed : 1660 deleted records
2017-08-05
Flux rep processed : 4117added records
Flux rep modified new processed : 2838added records
Flux rep partant processed : 986 deleted records
2017-08-06
2017-08-07
Flux rep processed : 391added records
Flux rep modified new processed : 697added records
Flux rep partant processed : 222 deleted records
2017-08-08
Flux rep processed : 4810added records
Flux rep modified new processed : 3721added records
Flux rep partant processed : 1156 deleted records
2017-08-09
Flux rep processed :

Flux rep partant processed : 123 deleted records
2017-10-03
Flux rep processed : 5107added records
Flux rep modified new processed : 4657added records
Flux rep partant processed : 923 deleted records
2017-10-04
Flux rep processed : 3491added records
Flux rep modified new processed : 3519added records
Flux rep partant processed : 1000 deleted records
2017-10-05
Flux rep processed : 2673added records
Flux rep modified new processed : 3432added records
Flux rep partant processed : 1045 deleted records
2017-10-06
Flux rep processed : 3708added records


b'Skipping line 1376: expected 47 fields, saw 56\n'


Flux rep modified new processed : 2598added records
Flux rep partant processed : 879 deleted records
2017-10-07
Flux rep processed : 5509added records
Flux rep modified new processed : 2955added records
Flux rep partant processed : 808 deleted records
2017-10-08
2017-10-09
Flux rep processed : 553added records
Flux rep modified new processed : 625added records
Flux rep partant processed : 154 deleted records
2017-10-10
Flux rep processed : 4517added records
Flux rep modified new processed : 4463added records
Flux rep partant processed : 1222 deleted records
2017-10-11
Flux rep processed : 4575added records
Flux rep modified new processed : 4092added records
Flux rep partant processed : 1086 deleted records
2017-10-12
Flux rep processed : 4317added records
Flux rep modified new processed : 3472added records
Flux rep partant processed : 949 deleted records
2017-10-13
Flux rep processed : 4262added records
Flux rep modified new processed : 4035added records
Flux rep partant processed : 97

...

loaded and uniformize
Old stock
Old Stock deleted : 106690 deleted records
Stock processed : 64650added records
2019-11-26
2019-11-27
2019-11-28
Flux rep processed : 13594added records
Flux rep modified new processed : 17753added records
Flux rep partant processed : 5222 deleted records
2019-11-29
Flux rep processed : 5000added records
Flux rep modified new processed : 5172added records
Flux rep partant processed : 1742 deleted records
2019-11-30
Flux rep processed : 1184added records
Flux rep modified new processed : 1687added records
Flux rep partant processed : 469 deleted records
2019-12-01
2019-12-02
2019-12-03
Flux rep processed : 5453added records
Flux rep modified new processed : 8123added records
Flux rep partant processed : 2054 deleted records
2019-12-04
Flux rep processed : 4754added records
Flux rep modified new processed : 8816added records
Flux rep partant processed : 2721 deleted records
2019-12-05
Flux rep processed : 6861added records
Flux rep modified new processed :

Flux rep partant processed : 2601 deleted records
2020-02-01
Flux rep processed : 3841added records
Flux rep modified new processed : 4761added records
Flux rep partant processed : 1584 deleted records
2020-02-02
2020-02-03
Flux rep processed : 318added records
Flux rep modified new processed : 997added records
Flux rep partant processed : 395 deleted records
2020-02-04
Flux rep processed : 3727added records
Flux rep modified new processed : 5749added records
Flux rep partant processed : 1890 deleted records
2020-02-05
Flux rep processed : 2800added records
Flux rep modified new processed : 3618added records
Flux rep partant processed : 1165 deleted records
2020-02-06
Flux rep processed : 6715added records
Flux rep modified new processed : 7554added records
Flux rep partant processed : 2399 deleted records
2020-02-07
Flux rep processed : 4719added records
Flux rep modified new processed : 6428added records
Flux rep partant processed : 1929 deleted records
2020-02-08
Flux rep processed 

Flux rep partant processed : 138 deleted records
2020-04-08
Flux rep processed : 2937added records
Flux rep modified new processed : 3862added records
Flux rep partant processed : 1317 deleted records
2020-04-09
Flux rep processed : 2715added records
Flux rep modified new processed : 3576added records
Flux rep partant processed : 1221 deleted records
2020-04-10
Flux rep processed : 1706added records
Flux rep modified new processed : 2562added records
Flux rep partant processed : 971 deleted records
2020-04-11
Flux rep processed : 1402added records
Flux rep modified new processed : 2018added records
Flux rep partant processed : 570 deleted records
2020-04-12
2020-04-13
Flux rep processed : 60added records
Flux rep modified new processed : 0added records
Flux rep partant processed : 0 deleted records
2020-04-14
Flux rep processed : 330added records
Flux rep modified new processed : 693added records
Flux rep partant processed : 301 deleted records
2020-04-15
Flux rep processed : 1295added

Flux rep processed : 5350added records
Flux rep modified new processed : 5554added records
Flux rep partant processed : 2118 deleted records
2020-06-17
Flux rep processed : 3630added records
Flux rep modified new processed : 4121added records
Flux rep partant processed : 1559 deleted records
2020-06-18
Flux rep processed : 3569added records
Flux rep modified new processed : 4023added records
Flux rep partant processed : 1522 deleted records
2020-06-19
Flux rep processed : 2971added records
Flux rep modified new processed : 3243added records
Flux rep partant processed : 1341 deleted records
2020-06-20
Flux rep processed : 3577added records
Flux rep modified new processed : 3576added records
Flux rep partant processed : 1298 deleted records
2020-06-21
2020-06-22
2020-06-23
Flux rep processed : 4052added records
Flux rep modified new processed : 4614added records
Flux rep partant processed : 1969 deleted records
2020-06-24
Flux rep processed : 4342added records
Flux rep modified new proce

Flux rep partant processed : 1638 deleted records
2020-09-17
Flux rep processed : 3176added records
Flux rep modified new processed : 3705added records
Flux rep partant processed : 1659 deleted records
2020-09-18
Flux rep processed : 4274added records
Flux rep modified new processed : 3795added records
Flux rep partant processed : 1492 deleted records
2020-09-19
Flux rep processed : 3232added records
Flux rep modified new processed : 3361added records
Flux rep partant processed : 1236 deleted records
2020-09-20
2020-09-21
2020-09-22
Flux rep processed : 3912added records
Flux rep modified new processed : 4121added records
Flux rep partant processed : 1624 deleted records
2020-09-23
Flux rep processed : 4297added records
Flux rep modified new processed : 4238added records
Flux rep partant processed : 1564 deleted records
2020-09-24
Flux rep processed : 3927added records
Flux rep modified new processed : 4031added records
Flux rep partant processed : 1477 deleted records
2020-09-25
Flux 

In [ ]:
conn = sqlite3.connect('inpi.db', isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)
print('connected')

db = pd.read_sql_query("SELECT * FROM rep_pm", conn)
print('read rep_pm ok')
db = db.drop_duplicates(keep="first")
print('deduplicates ok')
dbpm = db.groupby(['siren','denomination', 'siren_pm'],as_index=False)['qualite'].agg(lambda col: ', '.join(col))
print('groupby ok')
dbpm[dbpm['siren'] != ''].to_csv('rep_pm.csv',index=False)
print('csv saved ok')

db = pd.read_sql_query("SELECT * FROM rep_pp", conn)
print('read rep_pp ok')

inter = db[db['villenaissance'].notna()]
inter = inter[['siren', 'nom_patronymique', 'nom_usage', 'prenoms', 'villenaissance']]
inter = inter.drop_duplicates(keep="first")
db = db.drop(['villenaissance'], axis=1)
db = pd.merge(db, inter, on=['siren', 'nom_patronymique', 'nom_usage', 'prenoms'],how='left')

inter = db[db['paysnaissance'].notna()]
inter = inter[['siren', 'nom_patronymique', 'nom_usage', 'prenoms', 'paysnaissance']]
inter = inter.drop_duplicates(keep="first")
db = db.drop(['paysnaissance'], axis=1)
db = pd.merge(db, inter, on=['siren', 'nom_patronymique', 'nom_usage', 'prenoms'],how='left')

inter = db[db['datenaissance'].notna()]
inter = inter[['siren', 'nom_patronymique', 'nom_usage' 'prenoms', 'datenaissance']]
inter = inter.drop_duplicates(keep="first")
db = db.drop(['datenaissance'], axis=1)
db = pd.merge(db, inter, on=['siren', 'nom_patronymique', 'nom_usage', 'prenoms'],how='left')


db = db.drop_duplicates(keep="first")
print('deduplicates ok')
dbpp = db.groupby(['siren','nom_patronymique', 'nom_usage','prenoms', 'villenaissance', 'paysnaissance', 'datenaissance'],as_index=False)['qualite'].agg(lambda col: ', '.join(col))
print('groupby ok')
dbpp[dbpp['siren'] != ''].to_csv('rep_pp.csv',index=False)
print('csv saved ok')
